We need to break out columns into the pre-stop observables - e.g. reason for stop, age / weight / build, precinct, xcoord/ycoord
from the during-stop features

In [32]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import data_cleaner as dc
import data_modeler as dm

In [2]:
df = dc.load_full_sqf()

In [3]:
full_df = df.copy()
df = df.sample(5000)

In [5]:
df.describe()

,year,pct,ser_num,perobs,perstop,explnstp,othpers,arstmade,sumissue,compyear,...,premtype,state,addrpct,beat,xcoord,ycoord,linecm,detailcm,wepfound,detail1_
count,5000.000000,5000.000000,5000.000000,4985.000000,4999.000000,5000.000000,5000.000000,5000.000000,5000.000000,4977.0,...,0.0,0.0,4201.000000,1449.000000,4.051000e+03,4051.000000,3673.000000,4201.000000,5000.000000,528.000000
mean,2008.572000,67.954800,6015.843600,2.635850,5.521104,0.997000,0.233600,0.059200,0.068800,0.0,...,NaN,NaN,67.820757,8.021394,1.003828e+06,205340.278697,0.971958,38.875982,0.000800,37.301136
std,2.880985,33.199325,6048.825102,6.014787,6.103409,0.054696,0.423163,0.236022,0.253139,0.0,...,NaN,NaN,33.200308,6.515511,3.875688e+04,28963.999764,0.165117,26.067844,0.028276,25.790541
min,2003.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,NaN,NaN,1.000000,1.000000,5.933800e+05,121098.000000,0.000000,8.000000,0.000000,8.000000
25%,2006.000000,41.000000,1702.750000,1.000000,3.000000,1.000000,0.000000,0.000000,0.000000,0.0,...,NaN,NaN,41.000000,4.000000,9.961460e+05,183729.000000,1.000000,20.000000,0.000000,20.000000
50%,2009.000000,73.000000,4285.500000,1.000000,5.000000,1.000000,0.000000,0.000000,0.000000,0.0,...,NaN,NaN,73.000000,7.000000,1.005373e+06,200207.000000,1.000000,28.000000,0.000000,28.000000
75%,2011.000000,102.000000,8365.000000,2.000000,5.000000,1.000000,0.000000,0.000000,0.000000,0.0,...,NaN,NaN,101.000000,10.000000,1.017308e+06,232339.000000,1.000000,46.000000,0.000000,46.000000
max,2018.000000,123.000000,91411.000000,200.000000,300.000000,1.000000,1.000000,1.000000,1.000000,0.0,...,NaN,NaN,123.000000,62.000000,1.062915e+06,268642.000000,1.000000,113.000000,1.000000,112.000000


In [9]:
y = df.arstmade

# Modeling on the yes-no columns

In [48]:
X_y_n = df.select_dtypes(include=['int8']).drop(columns='arstmade')

In [49]:
split = dm.load_split(X_y_n, y)

## Balance classes

In [50]:
split['y_train'].value_counts(normalize=True)

0    0.941867
1    0.058133
Name: arstmade, dtype: float64

In [51]:
smote = SMOTE()
X_train_resampled, y_train_resampled = smote.fit_sample(split['X_train'], split['y_train']) 
y_train_resampled.value_counts()

1    3532
0    3532
Name: arstmade, dtype: int64

In [43]:
logit = LogisticRegression(solver='saga', penalty='l1', max_iter=4000)

In [52]:
logit.fit(X_train_resampled, y_train_resampled)
y_hat_train = logit.predict(split['X_train'])
residuals = np.abs(split['y_train'] - y_hat_train)
print(pd.Series(residuals).value_counts())
print(pd.Series(residuals).value_counts(normalize=True))

0    3406
1     344
Name: arstmade, dtype: int64
0    0.908267
1    0.091733
Name: arstmade, dtype: float64


In [53]:
pd.Series(y_hat_train).value_counts()

0    3304
1     446
dtype: int64

In [54]:
y_hat_test = logit.predict(split['X_test'])
residuals = np.abs(split['y_test'] - y_hat_test)
print(pd.Series(residuals).value_counts())
print(pd.Series(residuals).value_counts(normalize=True))

0    1131
1     119
Name: arstmade, dtype: int64
0    0.9048
1    0.0952
Name: arstmade, dtype: float64


In [55]:
logit.coef_

array([[-0.49144642, -1.46957627, -2.42386536, -0.80574694,  1.90023864,
         4.31529434,  4.30774961, -0.74686696,  5.29947862,  0.        ,
         0.        ,  1.6295691 ,  0.        ,  0.10195139, -0.12430863,
         0.        , -0.70115563,  0.        , -0.61178456,  0.        ,
         2.53323551,  0.        , -1.49778969, -1.40750321, -0.015672  ,
        -0.67855971, -0.4235051 , -1.76313173, -0.66321882, -0.80287405,
        -0.24625685, -0.73195728, -2.11418972, -1.84113804, -0.82595638,
        -0.33452918, -2.01869254, -0.47348931, -0.70723881, -1.24120284,
         0.01281269, -0.65584233, -2.29074084, -1.98967813, -1.63384225,
        -1.18490896, -0.52818738, -0.64484247, -1.16738197, -0.51512953,
        -3.2273605 , -3.36836732, -3.83719605, -1.35230389,  0.17073368,
        -0.93166047, -1.76038318,  0.        ]])

In [56]:
split['X_train'].columns

Index(['explnstp', 'othpers', 'sumissue', 'offunif', 'frisked', 'searched',
       'contrabn', 'adtlrept', 'pistol', 'riflshot', 'asltweap', 'knifcuti',
       'machgun', 'othrweap', 'pf_hands', 'pf_wall', 'pf_grnd', 'pf_drwep',
       'pf_ptwep', 'pf_baton', 'pf_hcuff', 'pf_pepsp', 'pf_other', 'radio',
       'ac_rept', 'ac_inves', 'rf_vcrim', 'rf_othsw', 'ac_proxm', 'rf_attir',
       'cs_objcs', 'cs_descr', 'cs_casng', 'cs_lkout', 'rf_vcact', 'cs_cloth',
       'cs_drgtr', 'ac_evasv', 'ac_assoc', 'cs_furtv', 'rf_rfcmp', 'ac_cgdir',
       'rf_verbl', 'cs_vcrim', 'cs_bulge', 'cs_other', 'ac_incid', 'ac_time',
       'rf_knowl', 'ac_stsnd', 'ac_other', 'sb_hdobj', 'sb_outln', 'sb_admis',
       'sb_other', 'rf_furt', 'rf_bulg', 'wepfound'],
      dtype='object')

In [58]:
coef = pd.DataFrame(logit.coef_)
coef.columns = split['X_train'].columns
coef.T

,0
explnstp,-0.491446
othpers,-1.469576
sumissue,-2.423865
offunif,-0.805747
frisked,1.900239
searched,4.315294
contrabn,4.307750
adtlrept,-0.746867
pistol,5.299479
riflshot,0.000000


In [60]:
df.groupby(by=['sumissue','arstmade']).count().iloc[:,0]

sumissue  arstmade
0         0           4372
          1            284
1         0            332
          1             12
Name: year, dtype: int64